PYMC4 adaptation of mixed model  

In [2]:
import pandas as pd
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.max_rows', 500)

from jupyter_dash import JupyterDash
import dash
from dash import dcc 
from dash import html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc

import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)

import matplotlib.pyplot as plt
import seaborn as sns
#sns.set()
import arviz as az
import theano.tensor as tt
import patsy

import os
import re
import glob
import random
import plotnine
from sklearn import preprocessing
from tqdm import tqdm

import plotnine 
from plotnine.data import economics 
from plotnine import * 
import plotly.express as px

from skimage import measure, restoration,morphology

%config InlineBackend.figure_format = 'retina'

In [7]:
import arviz as az
import theano.tensor as tt
import pymc as pm
import xarray as xr
import aesara
import aesara.tensor as at

from pymc import HalfCauchy, Model, Normal, sample

print(f"Running on PyMC v{pm.__version__}")

az.style.use("arviz-darkgrid")

RANDOM_SEED = 58
rng = np.random.default_rng(RANDOM_SEED)

Running on PyMC v4.0.1


In [5]:
path = r'E:\Elliot\classification\data\beyes_table'
tabel_samp = pd.read_csv(os.path.join(path,'SG_granularty_table_80opening_signale_reduction_25sample.csv'))
tabel_samp.head(1)
def add_class_randomise_and_pm_reconst(table_input, obser_number=10,show_class = False, vec=None):
    from sklearn import preprocessing
    tabel_5samp = table_input
    image_group =  tabel_5samp['image_name'].values
    le = preprocessing.LabelEncoder()
    # encoding
    # class_name_encoded=le.fit_transform(class_name).tolist()
    image_group_encoded=le.fit_transform(image_group).tolist()


    # adding a uneaqe class column 
    image_group =  tabel_5samp['image_name'].values.tolist()
    image_group_cls = []
    for i,img in enumerate(image_group):
        image_group_cls.append(img.split('_')[0]+ '_'+str(image_group_encoded[i]))
    image_group_cls    
    tabel_5samp['image_name_cls'] = image_group_cls

     # data sorting
    un_name = np.unique(tabel_5samp['image_name_cls']).tolist()
    import random
    random.shuffle(un_name)
    un_name

    def restructure_table(table, unique_name):
        table_temp = table.loc[lambda x: (x['image_name_cls'] == unique_name),:]
        return table_temp
    table_train = restructure_table(tabel_5samp, un_name[0])
    for i in tqdm(range(1,obser_number)):
        table_temp = restructure_table(tabel_5samp, un_name[i])
        table_train = pd.concat((table_train,table_temp))

    from sklearn import preprocessing
    table_sel = table_train
    # class_name =  table_sel['class'].values
    image_group =  table_sel['image_name'].values
    le = preprocessing.LabelEncoder()
    # encoding
    # class_name_encoded=le.fit_transform(class_name).tolist()
    image_group_encoded=le.fit_transform(image_group).tolist()
    table_dict={'image_signal':table_sel.open_image_signal.values.tolist(),
                'raius_list':table_sel.radius_list.values.tolist(),
                'image_group':image_group_encoded}
    table = pd.DataFrame(table_dict)
    table['image_group'] = pd.Categorical(table['image_group'], ordered=False)

    image_group =  table_sel['image_name'].values.tolist()
    image_group_cls = []
    for i,img in enumerate(image_group):
        if show_class:
            image_group_cls.append(img.split('_')[0])
        else:
            image_group_cls.append(img.split('_')[0]+ '_'+str(image_group_encoded[i]))
    image_group_cls    
    table['image_name_cls'] = image_group_cls
    table_vec_1 = table.loc[table['raius_list'].isin(vec),:]
    return table_vec_1

vec_2 = np.linspace(1,79,20,endpoint=True,dtype=int )    
table_draft = add_class_randomise_and_pm_reconst(tabel_samp,obser_number=40, show_class = True, vec = vec_2)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:00<00:00, 514.53it/s]


In [9]:
a = np.array(table_draft['image_group'])
_, idx = np.unique(a, return_index=True)
id__ = np.array(np.repeat(a[np.sort(idx)],20))
opening_opr__ = np.array(table_draft.raius_list.values) # 40*5
signal__ = np.array(table_draft.image_signal.values) # 40*5
id_paramaters__ = 40


print('id:{}'.format(len(id__))) #4000
print('opening_opr:{}'.format(len(opening_opr__))) # 40*5
print('signal:{}'.format(len(signal__)))

id:800
opening_opr:800
signal:800


In [12]:
with pm.Model() as simple_model_decay:
    id_paramaters = pm.Data("id_paramaters",id_paramaters__)
    id_ = pm.Data("id_",id__)
    opening_opr = pm.Data("opening_opr",opening_opr__)
    signal_ = pm.Data("signal_",signal__)
    
    a = pm.Normal('a',50,20,shape= id_paramaters__)
    b = pm.Exponential('b',0.3,shape= id_paramaters__)
    c = pm.Normal('c',5,10,shape= id_paramaters__)
    
    mu = a[id__] + c[id__] * at.exp(-b[id__] * opening_opr__)  # linear model
    sigma_within = pm.Exponential("sigma_within", 1.0)  # prior stddev within image
    signal = pm.Normal("signal", mu=mu, sigma=sigma_within, observed=signal__)  # likelihood
    
    trace_simple_model_decay = pm.sample(4000, tune=4000, target_accept=0.9)  # likelihood

ValueError: The `shape` parameter must be a tuple, TensorVariable, int or list. Actual: <class 'aesara.tensor.sharedvar.TensorSharedVariable'>

In [14]:
def model_factory(x,y, z, sig):
    with pm.Model() as model:
        a = pm.Normal('a',50,20,shape= y)
        b = pm.Exponential('b',0.3,shape= y)
        c = pm.Normal('c',5,10,shape= y)
    
        mu = a[z] + c[z] * at.exp(-b[z] * x)  # linear model
        sigma_within = pm.Exponential("sigma_within", 1.0)  # prior stddev within image
        signal = pm.Normal("signal", mu=mu, sigma=sigma_within, observed=sig)  # likelihood
    
        trace_simple_model_decay = pm.sample(4000, tune=4000, target_accept=0.9)
    return model

In [15]:
with model_factory(x=opening_opr__,
                   y=id_paramaters__,
                   z=id__,
                   sig=signal__) as train_model:
    train_trace = pm.sample()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [a, b, c, sigma_within]


Sampling 4 chains for 4_000 tune and 4_000 draw iterations (16_000 + 16_000 draws total) took 315 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [a, b, c, sigma_within]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 94 seconds.


In [16]:
# new data
vec_2 = np.linspace(1,79,20,endpoint=True,dtype=int )    
table_test = add_class_randomise_and_pm_reconst(tabel_samp,obser_number=3, show_class = True, vec = vec_2)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 200.54it/s]


In [18]:
a = np.array(table_test['image_group'])
_, idx = np.unique(a, return_index=True)
id_test = np.array(np.repeat(a[np.sort(idx)],20))
opening_opr__test = np.array(table_test.raius_list.values) # 40*5
signal__test = np.array(table_test.image_signal.values) # 40*5
id_paramaters__test = 3
print('id:{}'.format(len(id_test))) #4000
print('opening_opr:{}'.format(len(opening_opr__test))) # 40*5
print('signal:{}'.format(len(signal__test)))

id:60
opening_opr:60
signal:60


In [19]:
with model_factory(x=opening_opr__test,
                   y=id_paramaters__test,
                   z=id_test,
                   sig=signal__test) as test_model:
    # We first have to extract the learnt global effect from the train_trace
    df = pm.trace_to_dataframe(train_trace,
                               varnames=["a","b","c",'sigma_within',"signal"],
                               include_transformed=True)
    # We have to supply the samples kwarg because it cannot be inferred if the
    # input trace is not a MultiTrace instance
    ppc = pm.sample_posterior_predictive(trace=df.to_dict('records'),
                                         samples=len(df))

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [a, b, c, sigma_within]


Sampling 4 chains for 4_000 tune and 4_000 draw iterations (16_000 + 16_000 draws total) took 113 seconds.


AttributeError: module 'pymc' has no attribute 'trace_to_dataframe'

In [20]:
ppc

NameError: name 'ppc' is not defined